In [32]:
import camelot

In [33]:
import glob

In [34]:
directory = '/Users/joqsan/0-Excel/Reading_From_Folders/AFP/*.pdf'

In [35]:
directory

'/Users/joqsan/0-Excel/Reading_From_Folders/AFP/*.pdf'

In [36]:
files = [filename for filename in glob.glob(directory)]

In [37]:
files

['/Users/joqsan/0-Excel/Reading_From_Folders/AFP/Habitat_01.pdf',
 '/Users/joqsan/0-Excel/Reading_From_Folders/AFP/Habitat_02.pdf',
 '/Users/joqsan/0-Excel/Reading_From_Folders/AFP/Habitat_03.pdf',
 '/Users/joqsan/0-Excel/Reading_From_Folders/AFP/PlanVital_02.pdf',
 '/Users/joqsan/0-Excel/Reading_From_Folders/AFP/PlanVital_03.pdf',
 '/Users/joqsan/0-Excel/Reading_From_Folders/AFP/PlanVital_01.pdf',
 '/Users/joqsan/0-Excel/Reading_From_Folders/AFP/Cuprum_03.pdf',
 '/Users/joqsan/0-Excel/Reading_From_Folders/AFP/Cuprum_02.pdf',
 '/Users/joqsan/0-Excel/Reading_From_Folders/AFP/Cuprum_01.pdf',
 '/Users/joqsan/0-Excel/Reading_From_Folders/AFP/Provida_03.pdf',
 '/Users/joqsan/0-Excel/Reading_From_Folders/AFP/Provida_02.pdf',
 '/Users/joqsan/0-Excel/Reading_From_Folders/AFP/Provida_01.pdf',
 '/Users/joqsan/0-Excel/Reading_From_Folders/AFP/Capital_01.pdf',
 '/Users/joqsan/0-Excel/Reading_From_Folders/AFP/Modelo_01.pdf',
 '/Users/joqsan/0-Excel/Reading_From_Folders/AFP/Capital_02.pdf',
 '/Users

## Let's test AFP

In [38]:
import camelot
import pandas as pd
%matplotlib notebook
import re
import calendar
import locale
locale.setlocale(locale.LC_ALL,'es_ES.UTF-8')

'es_ES.UTF-8'

In [39]:
months = list(calendar.month_abbr)
months = list(map(lambda x: x.capitalize(), months))

In [40]:
area = ['21, 442, 133, 422', '19, 407, 394, 244', '397, 366, 779, 318']

In [41]:
def transpose_data(df, month, name):
    df = df.transpose()
    df.columns = df.iloc[0, :]
    df = df.drop(0, axis=0)
    df = df.set_index([[month], [name]])
    
    return df

In [42]:
def afp_data_cleaning(dfs):
    
    part1 = dfs[0].df.copy()
    period = part1.iloc[0, 1]
    month_str = re.match(r"(?P<month>[0-9]{2})", period).group('month')
    month_id = int(re.sub(r'0([0-9])', r'\1', month_str))
    month = months[month_id]
    
    
    part2 = dfs[1].df.copy()
    part2 = part2.drop([0, 1])
    part2.loc[:3, 0] = part2.loc[:3, 0].str.replace('Cotización', 'SC')
    part2.iloc[-1, 0] = 'Total A Pagar SC'
    part2.reset_index(drop=True, inplace=True)
    
    part3 = dfs[2].df.copy()
    afp_name = part3.iloc[-1, 0].split(' ')[-1]
    part3 = part3.drop([0, 1, 10, 13])
    part3.iloc[0, 0] = 'Total Imponible'
    part3.iloc[-1, 0] = 'Total A Pagar A AFP'
    
    afp = pd.concat([part3, part2], ignore_index=True)
    afp.iloc[:, [1]] = afp.iloc[:, [1]].apply(lambda x: x.str.replace('.', '')).astype(int)
    
    afp = transpose_data(afp, month, afp_name)
    
    return afp

In [43]:
afp1 = camelot.read_pdf(files[8], pages='1', flavor='stream', table_areas=area)

In [44]:
afp1 = afp_data_cleaning(afp1)

In [45]:
afp1

,,Total Imponible,Cotización Obligatoria,Seguro Invalidez y Sobrevivencia (SIS),Cotización Voluntaria (APVI),Depósito Convenido,Depósito en Cuenta de Ahorro,Aportes de Indemnización Sustitutiva,Cotización por Trabajos Pesados,APV Colectivo Empleador,APV Colectivo Trabajador,Monto Capitalización Voluntaria,Monto Ahorro Voluntario,Total A Pagar A AFP,SC Afiliados,SC Empleador,Total A Pagar SC
Ene,Cuprum,1304583,149244,19960,0,0,0,0,0,0,0,0,0,169204,3267,35870,39137


In [46]:
afp_list = []
for afp_file in files:
    afp_dfs = camelot.read_pdf(afp_file, pages='1', flavor='stream', table_areas=area)
    afp = afp_data_cleaning(afp_dfs)
    afp_list.append(afp)

In [47]:
df = pd.concat(afp_list)

In [48]:
df

,,Total Imponible,Cotización Obligatoria,Seguro Invalidez y Sobrevivencia (SIS),Cotización Voluntaria (APVI),Depósito Convenido,Depósito en Cuenta de Ahorro,Aportes de Indemnización Sustitutiva,Cotización por Trabajos Pesados,APV Colectivo Empleador,APV Colectivo Trabajador,Monto Capitalización Voluntaria,Monto Ahorro Voluntario,Total A Pagar A AFP,SC Afiliados,SC Empleador,Total A Pagar SC
Ene,Habitat,9154213,1031680,119080,0,0,0,0,0,0,0,0,0,1150760,11006,222482,233488
Feb,Habitat,9154213,1031680,119080,0,0,0,0,0,0,0,0,0,1150760,11006,222482,233488
Mar,Habitat,5672009,639235,68770,0,0,0,0,0,0,0,0,0,708005,11006,123837,134843
Feb,PlanVital,2234221,249338,34184,0,0,0,0,0,0,0,0,0,283522,0,67026,67026
Mar,PlanVital,2588517,288879,42590,0,0,0,0,0,0,0,0,0,331469,0,83508,83508
Ene,PlanVital,2197538,245245,33623,0,0,0,0,0,0,0,0,0,278868,0,65925,65925
Mar,Cuprum,1006648,115160,15402,0,0,0,0,0,0,0,0,0,130562,3488,26712,30200
Feb,Cuprum,1304583,149244,19960,0,0,0,0,0,0,0,0,0,169204,3267,35870,39137
Ene,Cuprum,1304583,149244,19960,0,0,0,0,0,0,0,0,0,169204,3267,35870,39137
Mar,Provida,4556034,521666,69709,72000,0,72000,0,0,0,0,0,0,735375,9347,143952,153299


In [49]:
import calendar

In [50]:
import locale
locale.setlocale(locale.LC_ALL,'es_ES.UTF-8')

'es_ES.UTF-8'

In [51]:
meses= list(calendar.month_abbr)

In [52]:
meses = list(map(lambda x: x.capitalize(), meses))

In [53]:
meses

['',
 'Ene',
 'Feb',
 'Mar',
 'Abr',
 'May',
 'Jun',
 'Jul',
 'Ago',
 'Sep',
 'Oct',
 'Nov',
 'Dic']

In [54]:
month_categories = pd.CategoricalIndex(df.index.get_level_values(0), categories=meses[1:], ordered=True)

In [55]:
df.index = [month_categories, df.index.get_level_values(1)]

In [56]:
df

,,Total Imponible,Cotización Obligatoria,Seguro Invalidez y Sobrevivencia (SIS),Cotización Voluntaria (APVI),Depósito Convenido,Depósito en Cuenta de Ahorro,Aportes de Indemnización Sustitutiva,Cotización por Trabajos Pesados,APV Colectivo Empleador,APV Colectivo Trabajador,Monto Capitalización Voluntaria,Monto Ahorro Voluntario,Total A Pagar A AFP,SC Afiliados,SC Empleador,Total A Pagar SC
Ene,Habitat,9154213,1031680,119080,0,0,0,0,0,0,0,0,0,1150760,11006,222482,233488
Feb,Habitat,9154213,1031680,119080,0,0,0,0,0,0,0,0,0,1150760,11006,222482,233488
Mar,Habitat,5672009,639235,68770,0,0,0,0,0,0,0,0,0,708005,11006,123837,134843
Feb,PlanVital,2234221,249338,34184,0,0,0,0,0,0,0,0,0,283522,0,67026,67026
Mar,PlanVital,2588517,288879,42590,0,0,0,0,0,0,0,0,0,331469,0,83508,83508
Ene,PlanVital,2197538,245245,33623,0,0,0,0,0,0,0,0,0,278868,0,65925,65925
Mar,Cuprum,1006648,115160,15402,0,0,0,0,0,0,0,0,0,130562,3488,26712,30200
Feb,Cuprum,1304583,149244,19960,0,0,0,0,0,0,0,0,0,169204,3267,35870,39137
Ene,Cuprum,1304583,149244,19960,0,0,0,0,0,0,0,0,0,169204,3267,35870,39137
Mar,Provida,4556034,521666,69709,72000,0,72000,0,0,0,0,0,0,735375,9347,143952,153299


In [57]:
df = df.sort_index(level=0)

In [58]:
df

0             Total Imponible Cotización Obligatoria  \
Ene Capital           1371956                 156952   
    Cuprum            1304583                 149244   
    Habitat           9154213                1031680   
    Modelo            4188146                 451063   
    PlanVital         2197538                 245245   
    Provida           3507125                 401566   
Feb Capital           2125687                 243179   
    Cuprum            1304583                 149244   
    Habitat           9154213                1031680   
    Modelo            4188146                 451063   
    PlanVital         2234221                 249338   
    Provida           3507125                 401566   
Mar Capital           2689619                 307692   
    Cuprum            1006648                 115160   
    Habitat           5672009                 639235   
    Modelo            6845862                 737300   
    PlanVital         2588517                 288879   
    Provida           4556034                 521666   

0             Seguro Invalidez y Sobrevivencia (SIS)  \
Ene Capital                                    30341   
    Cuprum                                     19960   
    Habitat                                   119080   
    Modelo                                     64079   
    PlanVital                                  33623   
    Provida                                    53660   
Feb Capital                                    32523   
    Cuprum                                     19960   
    Habitat                                   119080   
    Modelo                                     64079   
    PlanVital                                  34184   
    Provida                                    53660   
Mar Capital                                    41151   
    Cuprum                                     15402   
    Habitat                                    68770   
    Modelo                                    104742   
    PlanVital                                  42590   
    Provida                                    69709   

0             Cotización Voluntaria (APVI) Depósito Convenido  \
Ene Capital                              0                  0   
    Cuprum                               0                  0   
    Habitat                              0                  0   
    Modelo                               0                  0   
    PlanVital                            0                  0   
    Provida                          72000                  0   
Feb Capital                              0                  0   
    Cuprum                               0                  0   
    Habitat                              0                  0   
    Modelo                               0                  0   
    PlanVital                            0                  0   
    Provida                          72000                  0   
Mar Capital                              0                  0   
    Cuprum                               0                  0   
    Habitat                              0                  0   
    Modelo                               0                  0   
    PlanVital                            0                  0   
    Provida                          72000                  0   

0             Depósito en Cuenta de Ahorro  \
Ene Capital                              0   
    Cuprum                               0   
    Habitat                              0   
    Modelo                               0   
    PlanVital                            0   
    Provida                          72000   
Feb Capital                              0   
    Cuprum                               0   
    Habitat                              0   
    Modelo                               0   
    PlanVital                            0   
    Provida                          72000   
Mar Capital                         

In [59]:
gb = df.groupby(level=0)

In [60]:
gb.sum()

,Total Imponible,Cotización Obligatoria,Seguro Invalidez y Sobrevivencia (SIS),Cotización Voluntaria (APVI),Depósito Convenido,Depósito en Cuenta de Ahorro,Aportes de Indemnización Sustitutiva,Cotización por Trabajos Pesados,APV Colectivo Empleador,APV Colectivo Trabajador,Monto Capitalización Voluntaria,Monto Ahorro Voluntario,Total A Pagar A AFP,SC Afiliados,SC Empleador,Total A Pagar SC
Ene,21723561,2435750,320743,72000,0,72000,0,0,0,0,0,0,2900493,38327,587739,626066
Feb,22513975,2526070,323486,72000,0,72000,0,0,0,0,0,0,2993556,43625,607247,650872
Mar,23358689,2609932,342364,72000,0,72000,0,0,0,0,0,0,3096296,36414,651499,687913
Abr,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
May,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Jun,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Jul,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Ago,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Sep,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Oct,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [61]:
with pd.ExcelWriter('output.xlsx') as writer:
    df.to_excel(writer, sheet_name='AFP')